تنظیمات و import ها

In [7]:
import os
import gc
import numpy as np
import pandas as pd

# ========= PATHS =========
ORDERS_PATH   = "orders.csv"
PRIOR_PATH    = "order_products__prior.csv"
TRAIN_PATH    = "order_products__train.csv"
PRODUCTS_PATH = "products.csv"
AISLES_PATH   = "aisles.csv"
DEPTS_PATH    = "departments.csv"

# ========= SUBSET SETTINGS =========
TARGET_N_ORDERS = 15000
MAX_USERS       = 20000   # سقف تعداد کاربر برای انتخاب سفارش‌ها (برای کنترل حجم)

# ========= CHUNKING =========
CHUNK_SIZE = 500_000  # اگر RAM کم بود: 200_000

# ========= OUTPUT =========
OUT_DIR = "out"
os.makedirs(OUT_DIR, exist_ok=True)
CLEAN_ORDER_PRODUCTS_PATH = os.path.join(OUT_DIR, "order_products_clean.csv")
CLEAN_ORDERS_PATH         = os.path.join(OUT_DIR, "orders_clean.csv")


توابع کمکی (Chunk-friendly)

In [8]:
def downcast_int(df: pd.DataFrame, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast="integer")
    return df

def read_orders_subset(orders_path: str, target_n_orders: int, max_users: int) -> pd.DataFrame:
    """
    1) orders.csv رو با ستون‌های لازم می‌خونه
    2) یک subset از کاربران انتخاب می‌کنه
    3) از بین سفارش‌های آن کاربران، target_n_orders سفارش برمی‌داره
    """
    usecols = ["order_id", "user_id", "eval_set", "order_number", "order_dow", "order_hour_of_day"]
    orders = pd.read_csv(orders_path, usecols=usecols)

    # حذف null ها (طبق صورت مسئله)
    orders = orders.dropna()

    # فقط سفارش‌هایی که واقعاً basket دارند (prior/train)
    orders = orders[orders["eval_set"].isin(["prior", "train"])].copy()

    # کنترل اندازه با انتخاب subset از کاربران
    rng = np.random.default_rng(42)
    users = orders["user_id"].unique()
    if len(users) > max_users:
        sampled_users = rng.choice(users, size=max_users, replace=False)
        orders = orders[orders["user_id"].isin(sampled_users)].copy()

    # حالا از بین سفارش‌ها، target_n_orders برمی‌داریم
    # (ترجیحاً سفارش‌های اخیرتر هر کاربر را نگه داریم)
    orders = orders.sort_values(["user_id", "order_number"], ascending=[True, False])

    if len(orders) > target_n_orders:
        orders = orders.head(target_n_orders).copy()

    orders = downcast_int(orders, ["order_id", "user_id", "order_number", "order_dow", "order_hour_of_day"])
    return orders

def load_order_products_filtered(path: str, keep_order_ids: set, chunk_size: int) -> pd.DataFrame:
    """
    order_products فایل‌های prior/train بسیار بزرگ‌اند.
    این تابع با chunk می‌خواند و فقط order_id هایی که می‌خواهیم نگه می‌دارد.
    """
    usecols = ["order_id", "product_id", "add_to_cart_order", "reordered"]
    chunks = []
    for chunk in pd.read_csv(path, usecols=usecols, chunksize=chunk_size):
        chunk = chunk.dropna()
        chunk = chunk[chunk["order_id"].isin(keep_order_ids)].copy()
        chunks.append(chunk)
    if not chunks:
        return pd.DataFrame(columns=usecols)
    df = pd.concat(chunks, ignore_index=True)
    df = downcast_int(df, ["order_id", "product_id", "add_to_cart_order", "reordered"])
    return df


اجرای Task 1: ساخت دیتاست تمیز

In [9]:
# ---- STEP 1: orders subset ----
orders_sub = read_orders_subset(ORDERS_PATH, TARGET_N_ORDERS, MAX_USERS)
keep_order_ids = set(orders_sub["order_id"].tolist())

print("Selected orders:", len(orders_sub))
print("Selected users:", orders_sub["user_id"].nunique())
print("Eval_set counts:\n", orders_sub["eval_set"].value_counts())

# ---- STEP 2: load order_products from BOTH prior + train (chunked) ----
prior_sub = load_order_products_filtered(PRIOR_PATH, keep_order_ids, CHUNK_SIZE)
train_sub = load_order_products_filtered(TRAIN_PATH, keep_order_ids, CHUNK_SIZE)

print("prior_sub rows:", len(prior_sub))
print("train_sub rows:", len(train_sub))

order_products = pd.concat([prior_sub, train_sub], ignore_index=True)

# آزاد کردن حافظه
del prior_sub, train_sub
gc.collect()

print("order_products total rows:", len(order_products))
print("Unique orders in order_products:", order_products["order_id"].nunique())
print("Unique products:", order_products["product_id"].nunique())


Selected orders: 15000
Selected users: 939
Eval_set counts:
 eval_set
prior    14397
train      603
Name: count, dtype: int64
prior_sub rows: 137327
train_sub rows: 6096
order_products total rows: 143423
Unique orders in order_products: 15000
Unique products: 14340


Cleaning دقیق برای Instacart

In [10]:
# ---- STEP 3: Clean invalid rows ----

# 1) حذف null ها (قبلاً هم انجام شد ولی اینجا دوباره امن)
order_products = order_products.dropna()

# 2) "returned/negative quality" در Instacart ستون quantity ندارد.
# جایگزین منطقی: حذف رکوردهای خراب در add_to_cart_order (باید >=1 باشد)
order_products = order_products[order_products["add_to_cart_order"] >= 1].copy()

# 3) حذف قیمت‌های <=0: در Instacart قیمت نداریم → این مرحله N/A است.
# اما برای سازگاری، فرض می‌کنیم quantity=1 و داده منفی وجود ندارد.

# 4) حذف تکراری‌ها داخل هر سفارش (اگر به هر دلیل محصول دوبار آمده باشد)
order_products = order_products.drop_duplicates(subset=["order_id", "product_id"]).copy()

# 5) حذف سفارش‌هایی که فقط 1 کالا دارند (خیلی مهم برای ARM)
order_sizes = order_products.groupby("order_id")["product_id"].nunique()
valid_orders = order_sizes[order_sizes >= 2].index

order_products = order_products[order_products["order_id"].isin(valid_orders)].copy()
orders_sub = orders_sub[orders_sub["order_id"].isin(valid_orders)].copy()

print("After removing 1-item orders:")
print("orders_sub:", orders_sub.shape)
print("order_products:", order_products.shape)
print("Min basket size:", int(order_products.groupby("order_id")["product_id"].nunique().min()))


After removing 1-item orders:
orders_sub: (14132, 6)
order_products: (142555, 4)
Min basket size: 2


ذخیره خروجی برای Task 2

In [11]:
# ---- STEP 4: Save cleaned outputs ----
orders_sub.to_csv(CLEAN_ORDERS_PATH, index=False)
order_products.to_csv(CLEAN_ORDER_PRODUCTS_PATH, index=False)

print("Saved:")
print(" -", CLEAN_ORDERS_PATH)
print(" -", CLEAN_ORDER_PRODUCTS_PATH)


Saved:
 - out/orders_clean.csv
 - out/order_products_clean.csv


Self-Check نهایی

In [12]:
print("\n=== TASK 1 SELF-CHECK ===")
print("Clean orders:", orders_sub["order_id"].nunique())
print("Clean order_products rows:", len(order_products))
print("Unique products:", order_products["product_id"].nunique())

basket_sizes = order_products.groupby("order_id")["product_id"].nunique()
print("Basket size min:", int(basket_sizes.min()))
print("Basket size mean:", float(basket_sizes.mean()))
print("Basket size max:", int(basket_sizes.max()))



=== TASK 1 SELF-CHECK ===
Clean orders: 14132
Clean order_products rows: 142555
Unique products: 14304
Basket size min: 2
Basket size mean: 10.087390319841495
Basket size max: 74
